Модуль ipadress позволяет работать с ip-адресами, как с объектами

Мой айпишник - 178.176.164.201 узнать свой айпи можно с помощью сервиса: https://ifconfig.me

In [4]:
from ipaddress import IPv4Address

addr = IPv4Address("178.176.164.201")
addr

IPv4Address('178.176.164.201')

Данный модуль так же позволяет получать айпиадреса в виде int и байтовых значений

In [5]:
addr = IPv4Address(3691907365)  # From an int
addr

IPv4Address('220.14.9.37')

In [6]:
addr = IPv4Address(b"\xdc\x0e\t%")  # From bytes (packed form)
addr

IPv4Address('220.14.9.37')

Вышеупомянутые преобразования возможны и в другом направлении. Значит, мы можем распаковывать адрес в удобную нам форму:

In [7]:
int(addr)

3691907365

In [8]:
addr.packed

b'\xdc\x0e\t%'

Кроме того, что модуль ipadress позволяет работать с айпи адресом, как с классом с возможностью двустороннего ввода и вывода для различных типов Python, экземпляры класса IPv4Addressтакже могут быть хешированы, что дает возможность использовать их в качестве ключей в словаре:

In [9]:
hash(IPv4Address("220.14.9.37"))


-4203356137109932172

In [15]:
num_connections = {
     IPv4Address("220.14.9.37"): 2,
     IPv4Address("100.201.0.4"): 16,
     IPv4Address("8.240.12.2"): 4
}
num_connections

{IPv4Address('220.14.9.37'): 2,
 IPv4Address('100.201.0.4'): 16,
 IPv4Address('8.240.12.2'): 4}

Это означает, что их можно индексировать так:

In [ ]:
>>> net4[1]
IPv4Address('192.0.2.1')
>>> net4[-1]
IPv4Address('192.0.2.255')
>>> net6[1]
IPv6Address('2001:db8::1')
>>> net6[-1]
IPv6Address('2001:db8::ffff:ffff')

Вдобавок к этому в модуле IPv4Address реализованы методы, позволяющие проводить сравнения:

In [22]:
IPv4Address("220.14.9.37") > IPv4Address("8.240.12.2")

True

In [23]:
addrs = (
     IPv4Address("220.14.9.37"),
     IPv4Address("8.240.12.2"),
     IPv4Address("100.201.0.4"),
 )
for a in sorted(addrs):
    print(a)

8.240.12.2
100.201.0.4
220.14.9.37


Допустимо использовать любой из стандартных операторов сравнения целочисленных
значений обьектов класса: ==, !=, <, <=, >, >=

<h1> <b> Определение сетей </b> </h1>
Адреса узлов обычно группируются в IP-сети, поэтому модуль ipaddress предоставляет способ создания, проверки и управления определениями сетей. Объекты IP-сети создаются из строки, которые определяют диапазон адресов узлов, входящих в эту сеть. Простейшей формой для этой информации является пара «сетевой адрес/сетевой префикс», где префикс определяет количество ведущих битов, которые сравниваются для определения того, является ли адрес частью сети, и сетевой адрес определяет ожидаемое значение этих битов.

Что касается адресов, то предусмотрена функция, которая автоматически определяет правильную версию IP протокола:

In [47]:
>>> from ipaddress import IPv4Network
>>> ipaddress.ip_network('192.0.2.0/24')
IPv4Network('192.0.2.0/24')
>>> ipaddress.ip_network('2001:db8::0/96')

IPv6Network('2001:db8::/96')

Сетевые объекты не могут иметь заданные биты сетевого префикса. 
Суть в том, что 192.0.2.1/24 не описывает сеть. Такие определения упоминаются, поскольку IP адресов на сетевом уровне  не достаточно для описания сетевых интерфейсы компьютера в данной сети. Подробнее о подсетях поговорим далее.

По умолчанию попытка создать сетевой объект с содердащего биты хоста приведет к ValueError. Поэтому при обращении к обьекту в подсети необходимо, чтобы дополнительные биты были принудительно приравнены к  логическому нулю. Для этого флаг strict=False может быть передан  следующим образом:

In [ ]:
>>> ipaddress.ip_network('192.0.2.1/24')
Traceback (most recent call last):
   ...
ValueError: 192.0.2.1/24 has host bits set
>>> ipaddress.ip_network('192.0.2.1/24', strict=False)
IPv4Network('192.0.2.0/24')

Несмотря на то, что  форма строки предлагает значительно большую гибкость, сети также могут быть определены целыми числами, так же как адреса узлов. В этом случае считается, что сеть содержит только один адрес, идентифицированный целым числом, поэтому сетевой префикс включает в себя весь сетевой адрес:

In [ ]:
>>> ipaddress.ip_network(3221225984)
IPv4Network('192.0.2.0/32')
>>> ipaddress.ip_network(42540766411282592856903984951653826560)
IPv6Network('2001:db8::/128')

Как и в случае с адресами, создание определенного вида сети может быть принудительно выполнено путем прямого вызова конструктора класса вместо использования стандартной функции.

<h2> <b> Обозначение CIDR </b> </h2>
Сеть определяется с использованием сетевого адреса и префикса в нотации бесклассовой междоменной маршрутизации (CIDR) :



In [48]:
>>> from ipaddress import IPv4Network
>>> net = IPv4Network("192.4.2.0/24")
>>> net.num_addresses

256

Обозначение CIDR представляет сеть как <network_address>/<prefix>. Маршрутизации префикс (или длина префикса , или просто префикс ), который в данном случае 24, является счетчиком ведущих бит , используемых для ответа на вопросы , такие как ли определенный адрес является частью сети или сколько адреса находится в сети. (Здесь ведущие биты относятся к первым N битам, считая слева от целого числа в двоичном формате.)

In [26]:
>>> from ipaddress import IPv4Network
>>> net = IPv4Network("192.4.2.0/24")
>>> net.num_addresses

256

Вы можете найти префикс маршрутизации в .prefixlenсвойстве:

In [27]:
>>> net.prefixlen

24

Пример: входит ли адрес 168.4.3.12 в сеть 168.4.3.0/24?

Ответ: да, так как ведущие 24 бита адреса 168.4.3.12 – это первые три октета: 168.4.3. Последний октет соответствует последним 8 битам 32-битного IP-адреса.


In [55]:
>>> net = IPv4Network("168.4.3.0/24")

>>> IPv4Address("168.4.3.12") in net

True

Таже существует команда netmask для маскирования битов в сравниваемых адресах.


In [28]:
>>> net.netmask

IPv4Address('255.255.255.0')

<h><b>Извлечение IP-версии:</h></b>
Мы сравниваете начальные биты, чтобы определить, является ли адрес частью сети. Если начальные биты совпадают, то адрес является частью сети

Последние 8 бит 192.4.2.12 замаскированы (0) и игнорируются при сравнении. И снова модуль ipaddress избавляет нас от любого рода матемаических рассчетов:

In [ ]:
>>> net = IPv4Network("192.4.2.0/24")

>>> IPv4Address("192.4.2.12") in net
True
>>> IPv4Address("192.4.20.2") in net
False

Это стало возможным благодаря тому сокровищу, которое представляет собой перегрузки оператора, с помощью которого IPv4Network определяется, __contains__() чтобы разрешить проверку на членство (вхождение?) с использованием оператора in.

Широквещательный адрес - это единственный адрес, который может использоваться для связи со всеми хостами сети:

In [50]:
>>> net.network_address

IPv4Address('192.4.2.0')

Если вы подумаете об этом таким образом, вы увидите, как /24 префикс на самом деле переводится в истинное значение IPv4Address:

In [ ]:
>>> net.prefixlen
24
>>> net.netmask
IPv4Address('255.255.255.0')  # 11111111 11111111 11111111 00000000

Из этого следует, что мы можем построить сеть АЙПИ4 из двух адресов:

In [ ]:
>>> IPv4Network("192.4.2.0/255.255.255.0")

Выше 192.4.2.0- сетевой адрес, а 255.255.255.0 сетевая маска.

На другом конце спектра в сети находится ее конечный адрес или широковещательный адрес , который представляет собой единственный адрес, который можно использовать для связи со всеми хостами в ее сети:

In [51]:
>>> net.broadcast_address

IPv4Address('192.4.2.255')

<h><b>Получение сети из интерфейса:</h></b>

In [38]:
>>> from ipaddress import IPv4Network
>>> from ipaddress import IPv6Network
>>> addr4 = ipaddress.ip_address('192.0.2.1')
>>> addr6 = ipaddress.ip_address('2001:db8::1')
>>> addr6.version
>>> addr4.version


4

In [43]:
>>> from ipaddress import IPv4Network
>>> host4 = ipaddress.ip_interface('192.0.2.1/24')
>>> host4.network

IPv4Network('192.0.2.0/24')

In [45]:
>>> from ipaddress import IPv4Network
>>> host6 = ipaddress.ip_interface('2001:db8::1/96')
>>> host6.network

IPv6Network('2001:db8::/96')

<h><b>Определение количества отдельных адресов в сети:</h></b>

In [ ]:
>>> net4 = ipaddress.ip_network('192.0.2.0/24')
>>> net4.num_addresses
256
>>> net6 = ipaddress.ip_network('2001:db8::0/96')
>>> net6.num_addresses
4294967296

<h><b> Итерация через «используемые» адреса в сети:</h></b>

In [ ]:
>>> net4 = ipaddress.ip_network('192.0.2.0/24')
>>> for x in net4.hosts():
...     print(x)  
192.0.2.1
192.0.2.2
192.0.2.3
192.0.2.4
...
192.0.2.252
192.0.2.253
192.0.2.254

Точно так же net.hosts() возвращает генератор, который выдает адреса, показанные выше, за исключением сетевого и широковещательного адресов:

In [ ]:
>>> h = net.hosts()
>>> type(h)
<class 'generator'>
>>> next(h)
IPv4Address('192.4.2.1')
>>> next(h)
IPv4Address('192.4.2.2')

<h><b>Получение маски сети (т.е. установленных битов, соответствующих сетевому префиксу) или маски хоста (любых битов, не являющихся частью маски сети):</h></b>

In [32]:
net4 = ipaddress.ip_network('192.0.2.0/24')
net4.netmask

net4.hostmask

net6 = ipaddress.ip_network('2001:db8::0/96')
net6.netmask

net6.hostmask

IPv6Address('::ffff:ffff')

<h><b>Развертование или сжатие адреса:</h></b>

In [ ]:
>>> addr6.exploded
'2001:0db8:0000:0000:0000:0000:0000:0001'
>>> addr6.compressed
'2001:db8::1'
>>> net6.exploded
'2001:0db8:0000:0000:0000:0000:0000:0000/96'
>>> net6.compressed
'2001:db8::/96'

Хотя IPv4 не поддерживает развертование или сжатие, связанные объекты по-прежнему предоставляют соответствующие свойства, так что нейтральный к версии код может легко гарантировать, что будет использована наиболее лаконичная или наиболее детальная форма  для IPv6 адресов и при этом IPv4 адреса будут обработаны по-прежнему правильно.

<h1><b>Подсети</h1></b>

Подсеть – это часть IP-сети: :

In [ ]:
>>> small_net = IPv4Network("192.0.2.0/28")
>>> big_net = IPv4Network("192.0.0.0/16")
>>> small_net.subnet_of(big_net)
True
>>> big_net.supernet_of(small_net)
True

В коде выше small_net содержит 16 адресов, а big_net – 65 536

Если 24 бит не хватает для нумерации всех устройств в подсети, разделение на подсети путем увеличения длины префикса с 24 до 25 подразумевает смещение битов для разделения сети на меньшие части. Возможно, это немного математически сложно. К счастью, для IPv4Network это легко, потому что.subnets() возвращает итератор по подсетям:

В передаваемом subnets() аргументе можно задать, каким должен быть новый префикс. Более высокий префикс означает больше подсетей.

In [ ]:
>>> for sn in net.subnets(new_prefix=28):
...     print(sn)
...
200.100.10.0/28
200.100.10.16/28
200.100.10.32/28
...
200.100.10.208/28
200.100.10.224/28
200.100.10.240/28

<h2><b>Использование IP-адресов с другими модулями</h></b2>

Другие модули, использующие IP-адреса (например, socket), обычно не принимают объекты непосредственно из этого модуля. Вместо этого они должны быть приведны к целому числу или строке, которые будет способен принять другой модуль:

In [ ]:
>>> addr4 = ipaddress.ip_address('192.0.2.1')
>>> str(addr4)
'192.0.2.1'
>>> int(addr4)
3221225985

<h2><b>Получение дополнительных сведений при сбое создания сущности</h2></b>

Создавая объекты адреса/сети/интерфейса, используя фабричные функции, о любых ошибках будет сообщено как ValueError с универсальным сообщением об ошибке, в котором просто говорится, что содережимое не быол признанл объектом необходимого типа. Отсутствие четко определенной ошибки - то, почему необходимо знать, является ли значение предполагаемым обьектом IPv4 или IPv6, а для этого необходимо получить больше деталей о том, почему значение было отклонено.

Для поддержки сценариев использования, когда полезно иметь доступ к этой дополнительной детали, отдельные конструкторы класс фактически делят ValueError на подклассы ipaddress.AddressValueError и ipaddress.NetmaskValueError, чтобы точно указать, какая часть определения не была правильно проанализирована.

Сообщения об ошибках являются значительно более подробными при непосредственном использовании конструкторов классов. Например:

In [ ]:
>>> ipaddress.ip_address("192.168.0.256")
Traceback (most recent call last):
  ...
ValueError: '192.168.0.256' does not appear to be an IPv4 or IPv6 address
>>> ipaddress.IPv4Address("192.168.0.256")
Traceback (most recent call last):
  ...
ipaddress.AddressValueError: Octet 256 (> 255) not permitted in '192.168.0.256'

>>> ipaddress.ip_network("192.168.0.1/64")
Traceback (most recent call last):
  ...
ValueError: '192.168.0.1/64' does not appear to be an IPv4 or IPv6 network
>>> ipaddress.IPv4Network("192.168.0.1/64")
Traceback (most recent call last):
  ...
ipaddress.NetmaskValueError: '64' is not a valid netmask

Однако оба исключения модуля имеют ValueError в качестве родительского класса, поэтому, если нет необходимости в получении определенного типа ошибка, можно писать код, как показано ниже:

In [ ]:
try:
    network = ipaddress.IPv4Network(address)
except ValueError:
    print('address/netmask is invalid for IPv4:', address)


<h1><b>Компоновщик</h1></b>
Модуль ipaddress использует преимущества шаблона проектирования «Компоновщик». Класс IPv4Address представляет собой компоновщик, который оборачивает обычное целое число.

Каждый экземпляр IPv4Address имеет атрибут _ip, число типа int. Многие свойства и методы класса определяются значением этого атрибута:

In [19]:
addr = IPv4Address("220.14.9.37")
addr._ip
3691907365

3691907365

Атрибут _ip отвечает за создание int(addr). Цепочка вызовов выглядит следующим образом:

In [20]:
int(my_addr) -> my_addr._int() -> my_addr._ip

SyntaxError: invalid syntax (<ipython-input-20-de353b0a5ca0>, line 1)

Убедимся в  силе ._ip путем расширения класса IPv4Address:

In [21]:
from ipaddress import IPv4Address

class MyIPv4(IPv4Address):
    def __and__(self, other: IPv4Address):
        if not isinstance(other, (int, IPv4Address)):
            raise NotImplementedError
        return self.__class__(int(self) & int(other))

Добавление .__and__() позволяет использовать бинарный оператор &, чтобы применять маску к IP-адресу:

In [22]:
addr = MyIPv4("100.127.40.32")
mask = MyIPv4("255.192.0.0")  # Соответствует префиксу /10

addr & mask

addr & 0xffc00000  # hex-литерал для 255.192.0.0


MyIPv4('100.64.0.0')

Метод __and__() позволяет использовать либо другой IPv4Address, либо непосредственно int в качестве маски. Поскольку MyIPv4 является подклассом IPv4Address, проверка isinstance() в данном случае вернет True.

Помимо перегрузки оператора, есть возможность добавить новые свойства:

In [24]:
import re
from ipaddress import IPv4Address

class MyIPv4(IPv4Address):
    @property
    def binary_repr(self, sep=".") -> str:
        """Представляет IPv4 в виде 4 блоков по 8 бит."""
        return sep.join(f"{i:08b}" for i in self.packed)  # 8 строка

    @classmethod
    def from_binary_repr(cls, binary_repr: str):
        """Создает IPv4 из двоичного представления."""
        i = int(re.sub(r"[^01]", "", binary_repr), 2)  # 14 строка
        return cls(i)

В методе binary_repr (строка 8), используется .packed для преобразования IP-адреса в массив байтов, который затем форматируется, как строковое представление бинарной формы.

В from_binary_repr, вызов int(re.sub(r"[^01]", "", binary_repr), 2) (строка 14) состоит из двух частей:

1)удаление из входящей строки всего, кроме нулей и единиц;

2)анализ результата с помощью int(<string>, 2).

    
Методы binary_repr() и from_binary_repr() позволяют так же проводить двустороннюю конвертацию.

In [25]:
MyIPv4("220.14.9.37").binary_repr
MyIPv4("255.255.0.0").binary_repr  # Маска для префикса /16 

MyIPv4.from_binary_repr("11011100 00001110 00001001 00100101")
MyIPv4('220.14.9.37')

MyIPv4('220.14.9.37')

<h1>Задание</h1>

Так как в модуль встроены проверки корректности адресов, можно ими пользоваться, например, чтобы проверить, является ли адрес адресом сети или хоста. Попробуйте сделать это!